### Data Fetching

In [2]:
!pip install yfinance

In [3]:
import yfinance as yf
import pandas as pd
import requests
import bs4 as bs

In [27]:
resp = requests.get("http://en.wikipedia.org/wiki/List_of_S%26P_500_companies")

'''
soup = bs.BeautifulSoup(resp.text, "lxml")
table = soup.find("table", { "class": "wikitable sortable" })
tickers = []
for row in table.findAll("tr")[1:]:
  ticker = row.findAll("td")[0].text
  tickers.append(ticker.strip())'''

<Response [403]>

In [23]:
data = download_sp500_data()

Fetching S&P 500 ticker list...


HTTPError: HTTP Error 403: Forbidden

In [6]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Use requests with a proper User-Agent header to avoid being blocked
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Get the page content
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
response = requests.get(url, headers=headers)

# Parse the HTML
soup = BeautifulSoup(response.text, 'html.parser')

# Find the first table on the page (which should be the S&P 500 companies table)
table_html = str(soup.find('table', {'class': 'wikitable'}))

# Use pd.read_html on the extracted table HTML
table = pd.read_html(table_html)[0]



C:\Users\mohdz\AppData\Local\Temp\ipykernel_39332\3038375000.py:21: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(table_html)[0]


0       MMM
1       AOS
2       ABT
3      ABBV
4       ACN
       ... 
498     XYL
499     YUM
500    ZBRA
501     ZBH
502     ZTS
Name: Symbol, Length: 503, dtype: object

In [10]:
table['Symbol'].to_csv('sp500_symbols.csv', index=False)

### Reading tickers through yfinance and finding returns

In [31]:

def download_sp500_data():
    # Step 1: Get the S&P 500 ticker list from Wikipedia
    print("Fetching S&P 500 ticker list...")
    # Clean tickers (Yahoo uses '-' instead of '.' for symbols like BRK.B)
    tickers = pd.read_csv('sp500_symbols.csv')
    tickers = tickers['Symbol'].to_list()
    print(f"Found {len(tickers)} tickers. Starting download...")

    # Step 2: Download the data
    # We use yf.download because it's optimized for bulk requests
    # period="1y" gives you the last year of data; change to "max" for everything
    data = yf.download(tickers[:10], period="1y", threads=True)

    
    # Save the consolidated file
    #data.to_csv("sp10_historical_data.csv")
    
    return data

data = download_sp500_data()

Fetching S&P 500 ticker list...
Found 503 tickers. Starting download...


[*********************100%***********************]  10 of 10 completed


In [35]:
data.head()

Price            Close                                                  \
Ticker               A        ABBV         ABT         ACN        ADBE   
Date                                                                     
2025-02-18  134.120346  189.880234  127.782318  383.728485  464.109985   
2025-02-19  136.054749  190.944534  129.696426  386.362183  456.989990   
2025-02-20  135.697632  194.359970  130.717255  381.035400  454.690002   
2025-02-21  134.288986  195.521011  132.435028  360.658875  444.320007   
2025-02-24  134.199692  197.456100  132.464478  360.312317  444.420013   

Price                                                                 ...  \
Ticker            AES         AFL         AMD        AOS         MMM  ...   
Date                                                                  ...   
2025-02-18   9.733148  102.018951  114.279999  64.403069  146.294067  ...   
2025-02-19   9.837194  102.835655  114.690002  66.010216  147.482162  ...   
2025-02-20  10.092583  102.609329  114.169998  65.451630  146.382446  ...   
2025-02-21  10.016913  100.523308  110.839996  64.657860  142.356628  ...   
2025-02-24  10.139878  102.422371  108.110001  64.902855  142.847580  ...   

Price        Volume                                                         \
Ticker            A     ABBV      ABT      ACN     ADBE       AES      AFL   
Date                                                                         
2025-02-18  2121000  6065500  5116200  3701100  2973800  16920900  1635700   
2025-02-19  1738100  4888700  3806800  1438000  2965500  10035800  1579200   
2025-02-20   872700  5067000  5046900  2252400  2337600  13167600  1431400   
2025-02-21  1112200  5889400  7160100  3751700  3376000  19723000  3539600   
2025-02-24  2037500  6305000  5947200  2755800  2807900  10345900  2475400   

Price                                   
Ticker           AMD      AOS      MMM  
Date                                    
2025-02-18  31720300  1488200  2601400  
2025-02-19  28807700  2437500  3216900  
2025-02-20  27219600  1275200  2231500  
2025-02-21  41353700  1097800  4278500  
2025-02-24  29072200   926800  3447500  

[5 rows x 50 columns]

In [55]:
opens = data.xs('Open', level=0, axis=1)
closes = data.xs('Close', level=0, axis=1)
rets = (opens - closes) / opens
closes

Ticker,A,ABBV,ABT,ACN,ADBE,AES,AFL,AMD,AOS,MMM
Date,,,,,,,,,,
2025-02-18,134.120346,189.880234,127.782318,383.728485,464.109985,9.733148,102.018951,114.279999,64.403069,146.294067
2025-02-19,136.054749,190.944534,129.696426,386.362183,456.989990,9.837194,102.835655,114.690002,66.010216,147.482162
2025-02-20,135.697632,194.359970,130.717255,381.035400,454.690002,10.092583,102.609329,114.169998,65.451630,146.382446
2025-02-21,134.288986,195.521011,132.435028,360.658875,444.320007,10.016913,100.523308,110.839996,64.657860,142.356628
2025-02-24,134.199692,197.456100,132.464478,360.312317,444.420013,10.139878,102.422371,108.110001,64.902855,142.847580
...,...,...,...,...,...,...,...,...,...,...
2026-02-10,129.000000,222.440002,112.279999,240.860001,264.670013,16.250000,115.879997,213.570007,80.470001,172.655273
2026-02-11,128.899994,220.889999,113.589996,230.429993,257.160004,16.440001,116.250000,213.580002,80.220001,172.077866
2026-02-12,124.879997,227.500000,111.470001,222.050003,262.500000,16.200001,115.879997,205.940002,79.620003,173.830002


### Momentum with last year daily data 

In [57]:
lookback = 126
hold_out = 21

delayed_price = closes.shift(hold_out)
starting_price = closes.shift(hold_out + lookback)

mom_score = (delayed_price/starting_price) - 1
mom_score

Ticker,A,ABBV,ABT,ACN,ADBE,AES,AFL,AMD,AOS,MMM
Date,,,,,,,,,,
2025-02-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-02-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-02-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-02-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-02-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2026-02-10,0.209314,0.161762,-0.037664,-0.001388,-0.080914,0.167497,0.087023,0.387584,0.031336,0.069795
2026-02-11,0.237110,0.167191,-0.047518,0.003464,-0.107196,0.145578,0.071833,0.420200,0.041971,0.073022
2026-02-12,0.300477,0.192883,-0.050477,0.002644,-0.148965,0.091288,0.084429,0.420024,0.060309,0.091603


In [58]:
all_daily_ranks = mom_score.rank(axis=1, pct=True)
all_daily_ranks

Ticker,A,ABBV,ABT,ACN,ADBE,AES,AFL,AMD,AOS,MMM
Date,,,,,,,,,,
2025-02-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-02-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-02-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-02-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-02-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2026-02-10,0.9,0.7,0.2,0.3,0.1,0.8,0.6,1.0,0.4,0.5
2026-02-11,0.9,0.8,0.2,0.3,0.1,0.7,0.5,1.0,0.4,0.6
2026-02-12,0.9,0.8,0.2,0.3,0.1,0.6,0.5,1.0,0.4,0.7


In [73]:
current_top_5 = all_daily_ranks.iloc[-1].sort_values(ascending=False).head(3).index.to_list()
current_top_5

['AMD', 'A', 'ABBV']

In [78]:
def sp500_stockprice(period='1y', interval='1d'):
    '''
    Downloads S&P500 stock prices given a period and interval and returns the dataframe.
    '''

    # Reading ticker symbols and converting to list format
    tickers = pd.read_csv('sp500_symbols.csv')
    tickers = tickers['Symbol'].to_list()

    # Downloading data for each stock from yahoo finance.
    # Retrieves multi column dataframe with open,close,min,max,volume at first level and ticker labels at second level
    data = yf.download(tickers, period=period, interval=interval, threads=True)

    return data

def mom_returns(stocks_df, lookback, lag, num_of_stocks):

    closes = stocks_df.xs('Close', level=0, axis=1)

    lookback = lookback
    hold_out = lag
    
    delayed_price = closes.shift(hold_out)
    starting_price = closes.shift(hold_out + lookback)
    
    mom_score = (delayed_price/starting_price) - 1

    all_daily_ranks = mom_score.rank(axis=1, pct=True)

    long_pos = all_daily_ranks.iloc[-1].sort_values(ascending=False).head(num_of_stocks).index.to_list()
    short_pos = all_daily_ranks.iloc[-1].sort_values(ascending=False).tail(num_of_stocks).index.to_list()

    print('Go Long on ', long_pos)
    print('Go Short on ', short_pos)
    

In [79]:
data = sp500_stockprice()
mom_returns(data, 126, 21, 10)

[**********            20%                       ]  101 of 503 completed$BRK.B: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")
[**********************89%******************     ]  450 of 503 completed$BF.B: possibly delisted; no price data found  (period=1y)
[*********************100%***********************]  503 of 503 completed

2 Failed downloads:
['BRK.B']: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")
['BF.B']: possibly delisted; no price data found  (period=1y)


Go Long on  ['SNDK', 'WDC', 'MU', 'CIEN', 'TER', 'WBD', 'LRCX', 'STX', 'ALB', 'INTC']
Go Short on  ['PAYC', 'SMCI', 'GDDY', 'COIN', 'CHTR', 'TTD', 'FISV', 'BF.B', 'BRK.B', 'Q']
